[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/d_elliptic/elliptic.ipynb)

In [ ]:
# Necessity libraries
import numpy as np

# %matplotlib ipympl
# for interactive figures in Jupyter notebooks
# cannot have comments after it
import matplotlib.pyplot as plt
from math import *
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Elliptic equations

## Solving the Poisson equation (5-point Laplacian stencil)

We now solve the Poisson equation
$$
\nabla^2 u = f
$$
on the domain $\Omega = [0,1]^2$ using
$$
f(x,y) = -\exp \left( -3 \left( (x-0.3)^2 + (y-0.7)^2 \right) \right)
$$
and
$$
u = 0 \qquad \text{on $\partial \Omega$.}
$$

### Setting up the matrix and source term

In [ ]:
# Grid setup
m = 25
mm = m*m
h = 1.0/(m+1)

# Create derivative matrix and source term
d = np.zeros((mm, mm))
f = np.empty((mm))
hfac = 1/(h*h)
for i in range(m):
    y = (i+1)*h
    for j in range(m):
        ij = m*j+i

        # Construct 5-point Laplacian stencil
        d[ij, ij] = -4*hfac
        if i > 0:
            d[ij, ij-1] = hfac
        if i < m-1:
            d[ij, ij+1] = hfac
        if j > 0:
            d[ij, ij-m] = hfac
        if j < m-1:
            d[ij, ij+m] = hfac

        # Source term
        x = (j+1)*h
        f[ij] = -exp(-3*((x-0.3)**2+(y-0.7)**2))

### Visualize the derivative matrix

In [ ]:
plt.spy(d)
plt.show()

### Solve the linear system and plot

In [ ]:
# Solve the linear system
u = np.linalg.solve(d, f)

In [ ]:
%matplotlib ipympl

# Reconstruct full grid
ui = np.zeros((m+2, m+2)) # initial values
uu = np.zeros((m+2, m+2)) # numerical solution
for i in range(m):
    ui[i+1, 1:m+1] = f[i*m:(i+1)*m]
    uu[i+1, 1:m+1] = u[i*m:(i+1)*m]

# Plot using Matplotlib
xa = np.linspace(0, 1, m+2)
mgx, mgy = np.meshgrid(xa, xa)
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Plot initial values
# surf = ax.plot_surface(mgx, mgy, ui, cmap=cm.plasma, alpha=0.3,
#                        rstride=1, cstride=1, linewidth=0)
# Plot solution
surf = ax.plot_surface(mgx, mgy, uu, cmap=cm.plasma,
                       rstride=1, cstride=1, linewidth=0)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('u')
plt.show()

## Method of manufactured solution and global error

The code below solves the Poisson equation using the method of manufactured solutions with
$$
u_\text{manu}(x,y) = x(x-1) e^{xy} \sin \pi y
$$
corresponding to
$$
\begin{align*}
f(x,y) &= -e^{xy} \big( 2\pi(x-1)x^2 \cos \pi y \\
        &\phantom{=} + (2-2y+x((x-1)(x^2+y^2-\pi^2)+4y)\big) \sin \pi y
\end{align*}
$$
and boundary conditions $u=0$ on $\partial \Omega$. It measures the global error
for a variety of grid sizes.

### Streamline the Poisson solve and error computation

In [ ]:
# Calculates the error of the Poisson equation in several different norms
def poisson_error(m, results):
    mm = m*m
    h = 1.0/(m+1)

    # Create derivative matrix and source term
    d = np.zeros((mm, mm))
    f = np.empty((mm))
    hfac = 1/(h*h)
    for j in range(m):
        y = (j+1)*h
        for i in range(m):
            ij = j*m+i

            # Construct 5-point Laplacian stencil
            d[ij, ij] = -4*hfac
            if i > 0:
                d[ij, ij-1] = hfac
            if i < m-1:
                d[ij, ij+1] = hfac
            if j > 0:
                d[ij, ij-m] = hfac
            if j < m-1:
                d[ij, ij+m] = hfac

            # Source term
            x = (i+1)*h
            f[ij] = exp(x*y)*(2*pi*(x-1)*x*x*cos(pi*y)
                              + (2-2*y+x*((-1+x)*(-pi*pi+x*x)+4*y+(-1+x)*y*y))*sin(pi*y))

    # Solve the linear system
    u = np.linalg.solve(d, f)

    # Compute global error by subtracting the analytical solution
    for j in range(m):
        y = (j+1)*h
        for i in range(m):
            x = (i+1)*h
            u[j*m+i] -= x*(x-1)*exp(x*y)*sin(pi*y)

    # Return 2-norm and infinity norm
    print(m, h, h*np.linalg.norm(u), np.linalg.norm(u, np.inf))
    results.append((m, h, h*np.linalg.norm(u), np.linalg.norm(u, np.inf)))

# Restore print results
results_5pts = []

# Calculate the error for a range of grid sizes
for m in (7, 15, 31, 63, 95):
    poisson_error(m, results_5pts)

### Visualize global error in 2-norm and infinity norm

In [ ]:
%matplotlib inline

# Extract h values, 2-norm, and infinity norm from results
h_values_5pts = [r[1] for r in results_5pts]
two_norms_5pts = [r[2] for r in results_5pts]
infinity_norms_5pts = [r[3] for r in results_5pts]

# Plot the norms of global error
fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=300)
ax.loglog(h_values_5pts, two_norms_5pts, label='2-norm', marker='o', color='C0', mfc='none')
ax.loglog(h_values_5pts, infinity_norms_5pts, '--', label='Infinity norm', marker='s', color='C0', mfc='none')

# Formatting
ax.set_xlabel('Grid spacing ($h$)')
ax.set_ylabel('Global error')
ax.axis('equal')
ax.legend()
ax.grid(True, which="major", linestyle="--")
plt.show()

## Solving the Laplace equation (9-point Laplacian stencil)

The example below implements the 9-point stencil for the Laplace equation using the manufactured solution
$$
  u_\text{manu}(x,y) = \cos 3x \exp 3y
$$
with the Dirichlet boundary conditions $u=u_\text{manu}$ are applied on $\partial \Omega$.

The program computes the global error for a variety of grid sizes.

### Streamline the Laplace solve and error computation

In [ ]:
# Exact solution of the Laplace equation
def u_ex(x, y):
    return cos(3*x)*exp(3*y)

# Calculates the error of the Poisson equation in several different norms
def laplace_error(m, results):
    m2 = m+2
    mm = m2*m2
    h = 1.0/(m+1)

    # Create derivative matrix, starting with the diagonal terms already
    # present
    hfac = 1/(6*h*h)
    d = np.diag(-20*hfac*np.ones(mm))
    f = np.zeros((mm))
    for j in range(1, m+1):
        for i in range(1, m+1):
            ij = j*m2+i

            # Construct 9-point Laplacian stencil
            d[ij, ij-m2-1] = hfac
            d[ij, ij-m2] = 4*hfac
            d[ij, ij-m2+1] = hfac
            d[ij, ij-1] = 4*hfac
            d[ij, ij+1] = 4*hfac
            d[ij, ij+m2-1] = hfac
            d[ij, ij+m2] = 4*hfac
            d[ij, ij+m2+1] = hfac

    # Fill in bottom and top Dirichlet conditions
    for i in range(0, m2):
        f[i] = -20*hfac*u_ex(i*h, 0)
        f[i+m2*(m+1)] = -20*hfac*u_ex(i*h, 1)

    # Fill in the side Dirichlet conditions
    for j in range(1, m+1):
        f[j*m2] = -20*hfac*u_ex(0, j*h)
        f[j*m2+m+1] = -20*hfac*u_ex(1, j*h)

    # Solve the linear system
    u = np.linalg.solve(d, f)

    # Compute global error by subtracting the analytical solution
    for j in range(m2):
        for i in range(m2):
            u[j*m2+i] -= u_ex(i*h, j*h)

    # Return 2-norm and infinity norm
    print(m, h, h*np.linalg.norm(u), np.linalg.norm(u, np.inf))
    results.append((m, h, h*np.linalg.norm(u), np.linalg.norm(u, np.inf)))

# Restore print results
results_9pts = []

# Calculate the error for a range of grid sizes
for m in (7, 15, 31, 63, 95):
    laplace_error(m, results_9pts)

### Visualize global error in 2-norm and infinity norm

In [ ]:
# Extract h values, 2-norm, and infinity norm from results
h_values_9pts = [r[1] for r in results_9pts]
two_norms_9pts = [r[2] for r in results_9pts]
infinity_norms_9pts = [r[3] for r in results_9pts]

# Plot the norms of global error
fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=300)
# ax.loglog(h_values_5pts, two_norms_5pts, label='2-norm', marker='o', color='C0', mfc='none')
# ax.loglog(h_values_5pts, infinity_norms_5pts, '--', label='Infinity norm', marker='s', color='C0', mfc='none')
ax.loglog(h_values_9pts, two_norms_9pts, label='2-norm', marker='o', color='C1', mfc='none')
ax.loglog(h_values_9pts, infinity_norms_9pts, '--', label='Infinity norm', marker='s', color='C1', mfc='none')

# # Add three reference lines for slopes 2, 4, and 6
# h_ref = np.array(h_values_5pts)
# ax.loglog(h_ref, 0.1*h_ref**2, 'r-', label='$O(h^2)$')
# ax.loglog(h_ref, 0.01*h_ref**4, 'r:', label='$O(h^4)$')
# ax.loglog(h_ref, 0.01*h_ref**6, 'r', label='$O(h^6)$')

# Formatting
ax.set_xlabel('Grid spacing ($h$)')
ax.set_ylabel('Global error')
ax.axis('equal')
ax.legend(loc='upper left', frameon=True)
ax.grid(True, which="major", linestyle="--")
plt.show()

## Solving the Poisson equation with adjustments

The fourth-order method can be extended to the Poisson equation $\nabla^2 u=f$ where $f$ is non-zero. The numerical $f_{ij}$ is modified to
$$
f_{ij} = f(x_i,y_j) + \frac{h^2}{12} \nabla^2 f(x_i,y_j)
$$
and the additional term cancels out the leading order error. We demonstrate this using the example
of
$$
u_\text{manu}(x,y) = x^3(1-x)y(1-y),
$$
which has
$$
f(x,y) = 2(x-1)x^3+6x(2x-1)(y-1)y
$$
and
$$
\nabla^2 f (x,y) = 24(x(2x-1)+(y-1)y).
$$

The code also demonstrates this, using a 5-point Laplacian stencil for $\nabla^2 f$, and again achieving $O(h^4)$ error.

### Streamline the Poisson solve and error computation

In [ ]:
# Source term for Poisson equation
def ff(x,y):
    return 2*(x-1)*x**3+6*x*(2*x-1)*(y-1)*y

# Calculates the error of the Poisson equation in several different norms
def poisson_error(m, results):
    mm=m*m
    h=1.0/(m+1)

    # Create derivative matrix and source term
    d=np.zeros((mm,mm))
    f=np.empty((mm))
    hfac=1/(6*h*h)
    for j in range(m):
        y=(j+1)*h
        for i in range(m):
            ij=j*m+i

            # Construct 9-point Laplacian stencil 
            d[ij,ij]=-20*hfac
            if j>0:
                if i>0: d[ij,ij-m-1]=hfac
                d[ij,ij-m]=4*hfac
                if i<m-1: d[ij,ij-m+1]=hfac
            if j<m-1:
                if i>0: d[ij,ij+m-1]=hfac
                d[ij,ij+m]=4*hfac
                if i<m-1: d[ij,ij+m+1]=hfac
            if i>0: d[ij,ij-1]=4*hfac
            d[ij,ij]=-20*hfac
            if i<m-1: d[ij,ij+1]=4*hfac

            # Source term
            x=(i+1)*h
            f[ij]=ff(x,y)
            
            # Correction from nabla^2 f (analytical)
            # f[ij]+=h*h/12*(24*(x*(2*x-1)+(y-1)*y))

            # Correction from nabla^2 f (numerical)
            # f[ij]+=1/12.*(ff(x-h,y)+ff(x+h,y)+ff(x,y-h)+ff(x,y+h)-4*ff(x,y))

    # Solve the linear system
    u=np.linalg.solve(d,f)

    # Compute global error by subtracting the analytical solution 
    for j in range(m):
        y=(j+1)*h
        for i in range(m):
            x=(i+1)*h
            u[j*m+i]-=x*x*x*(x-1)*y*(y-1)

    # Return 2-norm and infinity norm
    print(m,h,h*np.linalg.norm(u),np.linalg.norm(u,np.inf))
    results.append((m,h,h*np.linalg.norm(u),np.linalg.norm(u,np.inf)))

# Store print results
results = []

# Calculate the error for a range of grid sizes
for m in (7,15,31,63,95):
    poisson_error(m, results)


### Visualize global error in 2-norm and infinity norm

In [ ]:
# Extract h values, 2-norm, and infinity norm from results
h_values = [r[1] for r in results]
two_norms = [r[2] for r in results]
infinity_norms = [r[3] for r in results]

# Plot the norms of global error
fig, ax = plt.subplots(1, 1, figsize=(4, 3), dpi=300)
# ax.loglog(h_values_5pts, two_norms_5pts, label='2-norm', marker='o', color='C0', mfc='none')
# ax.loglog(h_values_5pts, infinity_norms_5pts, '--', label='Infinity norm', marker='s', color='C0', mfc='none')
# ax.loglog(h_values_9pts, two_norms_9pts, label='2-norm', marker='o', color='C1', mfc='none')
# ax.loglog(h_values_9pts, infinity_norms_9pts, '--', label='Infinity norm', marker='s', color='C1', mfc='none')
ax.loglog(h_values, two_norms, label='2-norm', marker='o', color='C1', mfc='none')
ax.loglog(h_values, infinity_norms, '--', label='Infinity norm', marker='s', color='C1', mfc='none')

# # Add three reference lines for slopes 2 and 4
# h_ref = np.array(h_values)
# ax.loglog(h_ref, 0.1*h_ref**2, 'r-', label='$O(h^2)$')
# ax.loglog(h_ref, 0.01*h_ref**4, 'r:', label='$O(h^4)$')

# Formatting
ax.set_xlabel('Grid spacing ($h$)')
ax.set_ylabel('Global error')
ax.axis('equal')
ax.legend(loc='upper left', frameon=True)
ax.grid(True, which="major", linestyle="--")
plt.show()